In [63]:
from pyspark.sql import SparkSession

spark = SparkSession\
       .builder\
       .appName("test")\
       .getOrCreate()

In [92]:
df = spark.read.json("hdfs://namenode:8020/tmp/data/covid19")

In [93]:
df.printSchema()

root
 |-- offset: long (nullable = true)
 |-- partition: long (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- timestampType: long (nullable = true)
 |-- topic: string (nullable = true)
 |-- value: string (nullable = true)



In [97]:
df.select('value').show()

+--------------------+
|               value|
+--------------------+
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAzM...|
|eyJ1c2VySUQiOiAxN...|
|eyJ1c2VySUQiOiAzM...|
|eyJ1c2VySUQiOiAxN...|
|eyJ1c2VySUQiOiAzN...|
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAxO...|
|eyJ1c2VySUQiOiAxN...|
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAxM...|
|eyJ1c2VySUQiOiAzM...|
|eyJ1c2VySUQiOiA5N...|
|eyJ1c2VySUQiOiAyO...|
|eyJ1c2VySUQiOiAyM...|
|eyJ1c2VySUQiOiA1N...|
|eyJ1c2VySUQiOiA0O...|
+--------------------+
only showing top 20 rows

